# Heron Bench

目的：Heron Benchに親しむ

In [6]:
from datasets import load_dataset

dataset_name = "turing-motors/Japanese-Heron-Bench"
dataset = load_dataset(dataset_name)

In [7]:
dataset['train'][0]

{'image': <PIL.Image.Image image mode=RGB size=1920x1038>}

In [8]:
for i, sample in enumerate(dataset['train']):
    idx = i + 1
    # format to xxx.png
    idx = str(idx).zfill(3)
    sample['image'].save(f"tmp/{idx}.jpg")

[事実]

- データセットには画像のみ格納されている

[操作概要]

- 画像とtextをjoinして扱いやすいデータセットを作成した上で，llm-jpにアップロードする
- ただし今回はデータセットをllm-jp/以下に作成せず，Silviase/以下に作成する
- questions_ja.jsonlをベースとして，answers_gpt4.jsonl, claude3_0404_ja.jsonl, gemini_0404_ja.jsonl, gpt4v_0404_ja.jsonlを結合する．
- model_idとtextを結合するときにはquestion_idをキーとして結合する．
    - ただし，textはtextキーではなく，['answer'][f'{model_id}']キーとして格納する．

In [8]:
import pandas as pd
from datasets import Dataset, Image

df = pd.read_json('../data/japanese-heron-bench/questions_ja.jsonl', lines=True)
df['image'] = df['image'].apply(lambda x: f'tmp/{x}')
df['answer'] = [{} for _ in range(len(df))]

def add_answer(df, filename):
    # read jsonl file in heron/playground/data/japanese-heron-bench
    df_new = pd.read_json(f'../data/japanese-heron-bench/{filename}.jsonl', lines=True)
    model_id = df_new['model_id'][0].lstrip('models/')
    answer = df_new['text']
    # add answer to df['answer'][model_id] for each row
    for i, row in df.iterrows():
        df.at[i, 'answer'][model_id] = answer[i]

    return df

df = add_answer(df, 'answers_gpt4')
df = add_answer(df, 'claude3_0404_ja')
df = add_answer(df, 'gemini_0404_ja')
df = add_answer(df, 'gpt4v_0404_ja')
df

,question_id,image,category,image_category,context,text,answer
0,0,tmp/001.jpg,complex,anime,宮崎駿監督によるジブリ作品「君たちはどう生きるか」のワンシーンです。机には２枚の青色の鳥の羽...,この作品は著名な映画監督により制作されました。イラストから誰の作品だと考えられますか？,{'gpt-4-0125-preview': 'この作品は宮崎駿監督によるものだと考えられま...
1,1,tmp/001.jpg,complex,anime,宮崎駿監督によるジブリ作品「君たちはどう生きるか」のワンシーンです。机には２枚の青色の鳥の羽...,映像に写っている少年は何をしていますか？,{'gpt-4-0125-preview': '映像に写っている少年は、机に置かれた青色の鳥...
2,2,tmp/001.jpg,conv,anime,宮崎駿監督によるジブリ作品「君たちはどう生きるか」のワンシーンです。机には２枚の青色の鳥の羽...,鳥の羽の色は何色でしょうか？,"{'gpt-4-0125-preview': '鳥の羽の色は青色です。', 'claude-..."
3,3,tmp/001.jpg,conv,anime,宮崎駿監督によるジブリ作品「君たちはどう生きるか」のワンシーンです。机には２枚の青色の鳥の羽...,机の上には何本の鉛筆が置かれていますか？,{'gpt-4-0125-preview': '机の上には2本の鉛筆が置かれています。これら...
4,4,tmp/001.jpg,detail,anime,宮崎駿監督によるジブリ作品「君たちはどう生きるか」のワンシーンです。机には２枚の青色の鳥の羽...,このイラストについて詳しく説明してください。,{'gpt-4-0125-preview': 'このイラストでは、少年が机に座り、一枚の青い...
...,...,...,...,...,...,...,...
98,98,tmp/021.jpg,complex,traffic,走行中の車窓から雪に覆われた道路を見た冬の風景です。遠くには美しい羊蹄山がそびえ、青空が広が...,奥に見える山の名前はなんですか？,"{'gpt-4-0125-preview': '奥に見える山の名前は羊蹄山です。', 'cl..."
99,99,tmp/021.jpg,complex,traffic,走行中の車窓から雪に覆われた道路を見た冬の風景です。遠くには美しい羊蹄山がそびえ、青空が広が...,この道を運転する上で気をつけるべきことは何でしょうか？,{'gpt-4-0125-preview': 'この道を運転する上で、まず雪に覆われた道路の...
100,100,tmp/021.jpg,conv,traffic,走行中の車窓から雪に覆われた道路を見た冬の風景です。遠くには美しい羊蹄山がそびえ、青空が広が...,この場所における制限速度はいくつですか？,{'gpt-4-0125-preview': 'この場所の制限速度は40kmです。左側に制限...
101,101,tmp/021.jpg,conv,traffic,走行中の車窓から雪に覆われた道路を見た冬の風景です。遠くには美しい羊蹄山がそびえ、青空が広が...,現在地からニセコまで何kmでしょうか？,"{'gpt-4-0125-preview': '現在地からニセコまでは12kmです。', '..."


In [9]:
from huggingface_hub import HfApi, HfFolder

heron_bench_llmjp_hf = Dataset.from_pandas(df)
heron_bench_llmjp_hf = heron_bench_llmjp_hf.cast_column('image', Image())

# データセットをHugging Face Hubにプッシュする関数
def push_dataset_to_hub(dataset, repo_id):
    # データセットのプッシュ
    dataset.push_to_hub(repo_id)

# HfApiインスタンスを作成
api = HfApi()

# HfFolderからトークンを取得
token = HfFolder.get_token()

# データセットをプッシュするリポジトリID
repo_id = "Silviase/Japanese-Heron-Bench"

# データセットをHugging Face Hubにプッシュ
push_dataset_to_hub(heron_bench_llmjp_hf, repo_id)


Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.27s/it]


In [2]:
from datasets import load_dataset
dataset_from_hf = load_dataset("Silviase/Japanese-Heron-Bench")
dataset_from_hf['train'][0]

{'question_id': 0,
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1038>,
 'category': 'complex',
 'image_category': 'anime',
 'context': '宮崎駿監督によるジブリ作品「君たちはどう生きるか」のワンシーンです。机には２枚の青色の鳥の羽が置かれており、少年が小型ナイフでそのうちの一つを加工しています。他には竹の棒や赤いワイヤー、２本の鉛筆が机の左側に置かれており、右側には数冊の本といくつかの書類が置かれている。',
 'text': 'この作品は著名な映画監督により制作されました。イラストから誰の作品だと考えられますか？',
 'answer': {'claude-3-opus-20240229': 'この独特なイラストのスタイルから、宮崎駿監督の作品だと考えられます。宮崎監督は日本を代表するアニメーション映画監督で、「天空の城ラピュタ」や「もののけ姫」、「千と千尋の神隠し」など数々の名作を生み出してきました。\n\nこのイラストには、宮崎作品によく登場する細部まで丁寧に描かれた背景や、ファンタジー的な世界観が表現されています。机の上の本や羽ペンなどのディテールにも、宮崎監督らしい繊細なタッチが感じられます。\n\n全体的に温かみのある色使いと、ノスタルジックな雰囲気も宮崎作品の特徴と言えるでしょう。この一枚の絵から、',
  'gemini-1.0-pro-vision-latest': ' この作品は、日本のアニメ映画監督である宮崎駿氏が制作したものです。宮崎駿氏は、世界的に有名なアニメ映画監督であり、代表作には『風の谷のナウシカ』『もののけ姫』『千と千尋の神隠し』などがあります。宮崎駿氏の作品は、その美しい映像と深いストーリーで世界中の人々から愛されています。',
  'gpt-4-0125-preview': 'この作品は宮崎駿監督によるものだと考えられます。画像の説明から、細部への注意と物語の中での小さなアイテムの重要性を示すスタイルが宮崎駿の作品に特徴的です。また、「君たちはどう生きるか」というタイトルも、宮崎駿の作品のテーマに合致しています。',
  'gpt-4-vision